In [1]:
#Load cleaned_airline_data.csv into Spark Dataframe
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-delay').getOrCreate()
df = spark.read.csv('cleaned_airline_data_v5.csv', header = True, inferSchema = True)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/anderl23/hadoop-3.3.4/spark-3.3.1/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/anderl23/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2022-12-15T20:03:58,229 WARN [main] org.apache.spark.util.Utils - Your hostname, DESKTOP-SHBG2MR resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wifi0)
2022-12-15T20:03:58,241 WARN [main] org.apache.spark.util.Utils - Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-12-15T20:03:59,369 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.printSchema()

root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEPT_TIME: double (nullable = true)
 |-- DEPT_TIME: double (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: integer (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- FL_YEAR: integer (nullable = true)
 |-- FL_MONTH: integer (nullable = true)
 |-- FL_DOM: integer (

In [3]:
#uncomment to calculate row count
#Show number of rows in dataframe
df.count()

35841068

In [4]:
#show first five rows in dataframe. (note this has been transposed in order to easily view)
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00
OP_CARRIER,VX,VX,VX,VX,VX
OP_CARRIER_FL_NUM,108,114,11,121,124
ORIGIN,LAX,LAX,JFK,PHL,LAX
DEST,IAD,IAD,SFO,LAX,PHL
CRS_DEPT_TIME,700.0,2205.0,730.0,700.0,1100.0
DEPT_TIME,700.0,2204.0,729.0,700.0,1104.0
DEP_DELAY,0,-1,-1,0,4
TAXI_OUT,8,12,18,14,12
WHEELS_OFF,708.0,2216.0,747.0,714.0,1116.0


In [6]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ["OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST"]

OP_CARRIER_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_Index")
df_1 = OP_CARRIER_indexer.fit(df).transform(df)

onehotencoder_OP_CARRIER_vector = OneHotEncoder(inputCol="OP_CARRIER_Index", outputCol="OP_CARRIER_vec")
df_2 = onehotencoder_OP_CARRIER_vector.fit(df_1).transform(df_1)

OP_CARRIER_FL_NUM_indexer = StringIndexer(inputCol="OP_CARRIER_FL_NUM", outputCol="OP_CARRIER_FL_NUM_Index")
df_3 = OP_CARRIER_FL_NUM_indexer.fit(df_2).transform(df_2)

onehotencoder_OP_CARRIER_FL_NUM_vector = OneHotEncoder(inputCol="OP_CARRIER_FL_NUM_Index", outputCol="OP_CARRIER_FL_NUM_vec")
df_4 = onehotencoder_OP_CARRIER_FL_NUM_vector.fit(df_3).transform(df_3)

ORIGIN_indexer = StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_Index")
df_5 = ORIGIN_indexer.fit(df_4).transform(df_4)

onehotencoder_ORIGIN_vector = OneHotEncoder(inputCol="ORIGIN_Index", outputCol="ORIGIN_vec")
df_6 = onehotencoder_ORIGIN_vector.fit(df_5).transform(df_5)

DEST_indexer = StringIndexer(inputCol="DEST", outputCol="DEST_Index")
df_7 = DEST_indexer.fit(df_6).transform(df_6)

onehotencoder_DEST_vector = OneHotEncoder(inputCol="DEST_Index", outputCol="DEST_vec")
df_8 = onehotencoder_DEST_vector.fit(df_7).transform(df_7)
df_8.show(3)

2022-12-15T20:08:18,989 WARN [Thread-4] org.apache.spark.sql.catalyst.util.package - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-------------------+----------+-----------------+------+----+-------------+---------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------+--------+------+------+----------------+---------------+-----------------------+---------------------+------------+----------------+----------+----------------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEPT_TIME|DEPT_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|FL_YEAR|FL_MONTH|FL_DOM|FL_DOW|OP_CARRIER_Index| OP_CARRIER_vec|OP_CARRIER_FL_NUM_Index|OP_CARRIER_FL_NUM_vec|ORIGIN_I

In [7]:
vectorAssembler = VectorAssembler(inputCols = ['OP_CARRIER_vec', 'OP_CARRIER_FL_NUM_vec', 'ORIGIN_vec','DEST_vec',
                                               'TAXI_OUT', 'TAXI_IN', 'CANCELLED','DIVERTED','CRS_ELAPSED_TIME',
                                               'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'FL_YEAR', 'FL_MONTH',
                                               'FL_DOM', 'FL_DOW'], outputCol = 'features')
v_df = vectorAssembler.transform(df_8)
v_df = v_df.select(['features', 'ARR_DELAY'])
v_df.show(3)

+--------------------+---------+
|            features|ARR_DELAY|
+--------------------+---------+
|(7924,[15,178,717...|      -27|
|(7924,[15,689,717...|       -9|
|(7924,[15,1073,71...|      -26|
+--------------------+---------+
only showing top 3 rows



In [8]:
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [9]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='ARR_DELAY', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

2022-12-15T20:08:21,573 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1240.9 KiB


2022-12-15T20:14:41,644 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1242.1 KiB


2022-12-15T20:14:43,497 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1242.0 KiB


2022-12-15T20:16:27,926 WARN [Executor task launch worker for task 1.0 in stage 22.0 (TID 268)] dev.ludovic.netlib.InstanceBuilder$NativeBLAS - Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
2022-12-15T20:16:28,001 WARN [Executor task launch worker for task 1.0 in stage 22.0 (TID 268)] dev.ludovic.netlib.InstanceBuilder$NativeBLAS - Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


2022-12-15T20:20:15,961 WARN [Executor task launch worker for task 14.0 in stage 22.0 (TID 281)] org.apache.spark.storage.memory.MemoryStore - Not enough space to cache rdd_75_14 in memory! (computed 113.0 MiB so far)
2022-12-15T20:20:16,467 WARN [Executor task launch worker for task 14.0 in stage 22.0 (TID 281)] org.apache.spark.storage.BlockManager - Persisting block rdd_75_14 to disk instead.


...

2022-12-15T20:38:08,636 WARN [Executor task launch worker for task 25.0 in stage 62.0 (TID 1132)] org.apache.spark.storage.memory.MemoryStore - Not enough space to cache rdd_75_25 in memory! (computed 65.0 MiB so far)
2022-12-15T20:38:08,728 WARN [Executor task launch worker for task 24.0 in stage 62.0 (TID 1131)] org.apache.spark.storage.memory.MemoryStore - Not enough space to cache rdd_75_24 in memory! (computed 65.0 MiB so far)


2022-12-15T20:38:18,742 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1243.2 KiB


2022-12-15T20:38:22,270 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1246.4 KiB


2022-12-15T20:44:49,543 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1247.5 KiB


Coefficients: (7924,[0,1,2,3,5,6,7,8,9,10,11,12,14,15,16,17,7174,7175,7182,7183,7184,7185,7187,7188,7189,7190,7191,7193,7194,7195,7196,7200,7203,7543,7546,7547,7548,7549,7552,7554,7555,7558,7559,7560,7577,7750,7912,7913,7916,7917,7918,7919,7920,7921],[6.272537433773334,-3.705960270638793,-0.4593746255635295,-0.2152964124597678,0.7986381437778339,5.25879845315017,0.008884025169823872,-2.6176271432260556,-2.65277195151177,2.604362922884076,4.030152423498098,-2.2187722021360616,0.004947141666011967,0.017781861265594324,-2.0311774623319376,1.124235306514084,1.3100636487296098,2.471272063626226,-0.01066141753895997,-2.219679273646862,-1.5494986438191842,-1.681873550961087,-3.8795155199415627,-0.6646018705899781,-2.840406411261809,-8.958707122604292,-9.839252782508273,0.006298392450582942,-3.3521006981853585,-3.4911263856895944,-0.027687428933329823,1.244283628428662,1.2241527688836362,-0.3493155882209044,-0.12182269510896811,-2.3991354440349455,3.7776832469482406,-0.6020933725056239,-2.3804

In [10]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 40.077081
r2: 0.095762


In [11]:
train_df.describe().show()

2022-12-15T20:48:14,206 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1249.1 KiB


+-------+------------------+
|summary|         ARR_DELAY|
+-------+------------------+
|  count|          25094263|
|   mean| 5.127828141436152|
| stddev|42.145850612557226|
|    min|              -238|
|    max|              2692|
+-------+------------------+



In [12]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","ARR_DELAY","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="ARR_DELAY",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

2022-12-15T20:53:51,521 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1239.0 KiB


+------------------+---------+--------------------+
|        prediction|ARR_DELAY|            features|
+------------------+---------+--------------------+
|1.0506625996729326|      -11|(7924,[0,19,7177,...|
| 3.241952133061659|      -14|(7924,[0,19,7177,...|
| 2.974165315333721|       -9|(7924,[0,19,7177,...|
|  8.87288815147258|      -12|(7924,[0,19,7177,...|
|-5.683861224159955|      -11|(7924,[0,19,7179,...|
+------------------+---------+--------------------+
only showing top 5 rows

2022-12-15T20:54:10,819 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1247.0 KiB


2022-12-15T20:59:50,849 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1248.1 KiB
R Squared (R2) on test data = 0.0953659


In [13]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

2022-12-15T20:59:59,208 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1246.8 KiB


2022-12-15T21:05:46,709 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1247.9 KiB
Root Mean Squared Error (RMSE) on test data = 40.1314


In [14]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 10
objectiveHistory: [0.49999998007511887, 0.48986914389878855, 0.4669502038466114, 0.46698181446507875, 0.46523675455697294, 0.4651261619426882, 0.4647588589230092, 0.46445430639189894, 0.46164934972839905, 0.46161606907324493, 0.4614816236660369]
2022-12-15T21:05:51,482 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1243.5 KiB


+-------------------+
|          residuals|
+-------------------+
|-16.922398515348732|
|-13.310532927567465|
|-12.617082699802268|
|-15.227951231503312|
|-15.753894749321262|
| -8.959789450942708|
|-13.128786630106788|
|-14.334315751009171|
|-15.498361851406116|
|-15.525230321565232|
|-14.379504726132382|
|-16.269314016357384|
|-16.426440683001374|
| -10.02714210017939|
|  -17.8180517459964|
|-14.366167382387857|
| -8.446509277375071|
| -5.236779003370884|
| -11.52103642138104|
| 57.594223817465036|
+-------------------+
only showing top 20 rows



In [15]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","ARR_DELAY","features").show()

2022-12-15T21:06:41,208 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1239.0 KiB


+-------------------+---------+--------------------+
|         prediction|ARR_DELAY|            features|
+-------------------+---------+--------------------+
| 1.0506625996729326|      -11|(7924,[0,19,7177,...|
|  3.241952133061659|      -14|(7924,[0,19,7177,...|
|  2.974165315333721|       -9|(7924,[0,19,7177,...|
|   8.87288815147258|      -12|(7924,[0,19,7177,...|
| -5.683861224159955|      -11|(7924,[0,19,7179,...|
| -3.369258685214163|       -6|(7924,[0,19,7179,...|
| -3.181309525724373|        8|(7924,[0,19,7179,...|
| -4.914501937503246|       -1|(7924,[0,19,7179,...|
|  1.252993091124381|       11|(7924,[0,19,7179,...|
|  4.249396204696495|        2|(7924,[0,19,7179,...|
| 2.0468597035478524|      -15|(7924,[0,19,7179,...|
| 0.7031737229008286|      -13|(7924,[0,19,7179,...|
| 2.8418418223557183|        7|(7924,[0,19,7179,...|
|  3.769920653950976|        2|(7924,[0,19,7179,...|
| -4.320928385468733|      -16|(7924,[0,19,7181,...|
|-1.9549500505970627|       -6|(7924,[0,19,718